In [1]:
import pandas as pd
import pickle
import networkx as nx
from subpred.util import save_data
from subpred.fasta import read_fasta
import re
import numpy as np

In [2]:
DATASET_FOLDER= "../data/datasets/"
RAW_DATA_FOLDER= "../data/raw/"

## Uniprot

In [3]:
names = [
    "Uniprot",
    "gene_names",
    "protein_names",
    "reviewed",
    "protein_existence",
    "sequence",
    "organism_id",
    "go_ids",
    "keyword_ids",
    "keywords",
    "tcdb_ids",
    "interpro_ids",
]
dtypes = {
    "Uniprot": "string",
    "gene_names": "string",
    "protein_names": "string",
    "reviewed": "category",
    "protein_existence": "category",
    "sequence": "string",
    "organism_id": "int",
    "go_ids": "string",
    "keyword_ids": "string",
    "keywords": "string",
    "tcdb_ids": "string",
    "interpro_ids": "string",
}

df = pd.read_table(
    RAW_DATA_FOLDER+"uniprot/uniprot_evidence1-2_nofragments.tsv",
    index_col=0,
    header=None,
    names=names,
    dtype=dtypes,
    skiprows=1,
)

In [4]:
df.reviewed = df.reviewed.transform(lambda x: x == "reviewed").astype("bool")

In [5]:
df.protein_existence = df.protein_existence.map(
    {"Evidence at transcript level": 2, "Evidence at protein level": 1}
).astype("int")

In [6]:
# sequences_nonstandard_aa_count = (
#     df[~df.sequence.str.fullmatch(re.compile("[ACDEFGHIKLMNPQRSTVWY]+"))]
#     .sequence.drop_duplicates()
#     .shape[0]
# )
# sequences_count = df.sequence.drop_duplicates().shape[0]
# print(
#     f"proteins with non-standard amino acids: {sequences_nonstandard_aa_count} out of {sequences_count} ({sequences_nonstandard_aa_count / sequences_count * 100:.2f}%)"
# )

In [7]:
# saving accessions of those proteins, for later stats

df_sequences_nonstandard = df[
    ~df.sequence.str.fullmatch(re.compile("[ACDEFGHIKLMNPQRSTVWY]+"))
].sequence.to_frame().drop_duplicates()
save_data(df_sequences_nonstandard, "proteins_nonstandard_aminoacids", folder_path=DATASET_FOLDER)


In [8]:
# removing the AAs (all features are based on 20 AAs)
df.sequence = df.sequence.str.replace(
    re.compile("[^ACDEFGHIKLMNPQRSTVWY]+"), "", regex=True
)

In [9]:
df_sequences = df.drop(
    ["go_ids", "keyword_ids", "keywords", "tcdb_ids", "interpro_ids"], axis=1
)
save_data(df_sequences, "uniprot", DATASET_FOLDER)
df_sequences.head()

,gene_names,protein_names,reviewed,protein_existence,sequence,organism_id
Uniprot,,,,,,
A0A0C5B5G6,MT-RNR1,Mitochondrial-derived peptide MOTS-c (Mitochon...,True,1,MRWQEMGYIFYPRKLR,9606
A0A1B0GTW7,CIROP LMLN2,Ciliated left-right organizer metallopeptidase...,True,1,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,9606
A0JNW5,BLTP3B KIAA0701 SHIP164 UHRF1BP1L,Bridge-like lipid transfer protein family memb...,True,1,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,9606
A0JP26,POTEB3,POTE ankyrin domain family member B3,True,1,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,9606
A0PK11,CLRN2,Clarin-2,True,1,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,9606


In [10]:
df_keywords = (
    df.keywords.dropna()
    .str.split(";")
    .explode()
    .str.strip()
    .astype("category")
    .rename("keyword")
    .to_frame()
    .reset_index(drop=False)
    .drop_duplicates()
    .reset_index(drop=True)
)
save_data(df_keywords, "keywords", DATASET_FOLDER)
df_keywords.head()

,Uniprot,keyword
0,A0A0C5B5G6,DNA-binding
1,A0A0C5B5G6,Mitochondrion
2,A0A0C5B5G6,Nucleus
3,A0A0C5B5G6,Osteogenesis
4,A0A0C5B5G6,Reference proteome


In [11]:
df_interpro = (
    df.interpro_ids.dropna()
    .str.rstrip(";")
    .str.split(";")
    .explode()
    .rename("interpro_id")
    .to_frame()
    .reset_index(drop=False)
    .drop_duplicates()
    .reset_index(drop=True)
)
df_interpro.head()

,Uniprot,interpro_id
0,A0A0C5B5G6,IPR054130
1,A0A1B0GTW7,IPR001577
2,A0JNW5,IPR026728
3,A0JNW5,IPR026854
4,A0JP26,IPR050657


In [12]:
df_interpro_annotations = pd.read_table(
    RAW_DATA_FOLDER+"interpro/interpro_entries.tsv",
    names=["accession", "type", "name"],
    skiprows=1,
    index_col=0,
    dtype={"accession": str, "type": "category", "name": str},
)
df_interpro_annotations.head()

,type,name
accession,,
IPR000126,Active_site,"Serine proteases, V8 family, serine active site"
IPR000138,Active_site,"Hydroxymethylglutaryl-CoA lyase, active site"
IPR000169,Active_site,"Cysteine peptidase, cysteine active site"
IPR000180,Active_site,"Membrane dipeptidase, active site"
IPR000189,Active_site,"Prokaryotic transglycosylase, active site"


In [13]:
df_interpro = df_interpro.merge(
    df_interpro_annotations, how="left", left_on="interpro_id", right_index=True
).drop_duplicates()
save_data(df_interpro, "interpro", DATASET_FOLDER)
df_interpro.head()

,Uniprot,interpro_id,type,name
0,A0A0C5B5G6,IPR054130,Family,Mitochondrial-derived peptide MOTS-c
1,A0A1B0GTW7,IPR001577,Family,"Peptidase M8, leishmanolysin"
2,A0JNW5,IPR026728,Family,Bridge-like lipid transfer protein family memb...
3,A0JNW5,IPR026854,Domain,Vacuolar protein sorting-associated protein 13...
4,A0JP26,IPR050657,Family,Ankyrin repeat domain-containing protein


In [14]:
df_tcdb_uniprot = (
    df.tcdb_ids.dropna()
    .str.rstrip(";")
    .str.split(";")
    .explode()
    .rename("tcdb_id")
    .reset_index(drop=False)
    .drop_duplicates()
    .melt(id_vars=["tcdb_id"], value_vars=["Uniprot"])
)
df_tcdb_uniprot = df_tcdb_uniprot[["tcdb_id", "value"]].drop_duplicates().reset_index(
    drop=True
)
save_data(df_tcdb_uniprot, "tcdb", DATASET_FOLDER)
df_tcdb_uniprot.head()

,tcdb_id,value
0,1.R.2.1.1,A0JNW5
1,9.A.46.1.2,A0PK11
2,9.B.438.1.1,A2RU14
3,1.N.2.1.1,A6NI61
4,9.A.80.1.1,A6NKB5


## 3Di

In [15]:
def process_fasta(fasta_data_3di: list, lowercase: bool = True):
    accession_pattern = re.compile(
        "[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}"
    )
    not_in_alphabet_pattern = re.compile("[^ACDEFGHIKLMNPQRSTVWYX]+")
    for header, sequence in fasta_data_3di:
        header_values = header[1:].split("-")
        accession = header_values[1]
        # number of the split, if more than 2700AA:
        fold_number = int(header_values[2][1:])
        if not re.fullmatch(accession_pattern, accession):
            print(
                "found invalid header in 3di fasta file, unable to read accession: "
                + header
            )
            continue

        if re.search(not_in_alphabet_pattern, sequence):
            print("Invalid characer found in 3di sequence " + sequence)
            continue

        # compatibility with prostt5
        # print(sequence_lower)
        yield accession, fold_number, sequence.lower() if lowercase else sequence


def merge_subsequences(
    df_uniprot_3Di: pd.DataFrame, get_stats: bool = True, overlap_len: int = 1200
):
    # In the case of proteins longer than 2700 amino acids (aa), AlphaFold provides 1400aa long, overlapping fragments.
    # For example, Titin has predicted fragment structures named as Q8WZ42-F1 (residues 1–1400), Q8WZ42-F2 (residues 201–1600), etc.
    # These fragments are currently only available for the human proteome in these proteome archive files, not on the website.
    # All structures seem to have overlap of 1200, might change in future versions
    df_uniprot_3Di_copy = df_uniprot_3Di.copy(deep=True)
    # proteins that were split into more than one structure
    split_proteins = df_uniprot_3Di_copy[
        df_uniprot_3Di_copy.fold_number == 2
    ].Uniprot.unique()

    records_merged_sequences = list()
    records_merging_stats = list()

    for split_protein in split_proteins:
        df_uniprot_3Di_split_protein = df_uniprot_3Di_copy[
            df_uniprot_3Di_copy.Uniprot == split_protein
        ][["fold_number", "sequence3Di"]].sort_values("fold_number")
        subsequences_list = list(
            df_uniprot_3Di_split_protein.sequence3Di.values.ravel()
        )
        sequence_merged = subsequences_list[0]
        sequence_identity_scores = list()
        for subsequence in subsequences_list[1:]:
            matching_pos = np.equal(
                list(subsequence[:overlap_len]), list(sequence_merged[-overlap_len:])
            )
            overlap_seqence_identity = sum(matching_pos) / len(matching_pos)
            sequence_identity_scores.append(overlap_seqence_identity)
            additional_positions = subsequence[overlap_len : len(subsequence)]
            sequence_merged += additional_positions

        records_merging_stats.append(
            [
                split_protein,
                sequence_identity_scores,
            ]
        )

        records_merged_sequences.append([split_protein, sequence_merged])

    df_merged = pd.DataFrame.from_records(
        records_merged_sequences, columns=["Uniprot", "sequence3Di"]
    )
    df_uniprot_3Di_copy = df_uniprot_3Di_copy[
        ~df_uniprot_3Di_copy.Uniprot.isin(df_merged.Uniprot)
    ]
    # any unmerged remain?
    assert (df_uniprot_3Di_copy.fold_number == 1).all()
    df_uniprot_3Di_copy = df_uniprot_3Di_copy.drop("fold_number", axis=1)

    df_uniprot_3Di_copy = pd.concat(
        [df_merged, df_uniprot_3Di_copy], ignore_index=True
    ).set_index("Uniprot")
    df_uniprot_3Di_merging_stats = pd.DataFrame.from_records(
        records_merging_stats,
        columns=[
            "Uniprot",
            "merging_overlaps_seq_ident",
        ],
        index="Uniprot",
    )
    if get_stats:
        return df_uniprot_3Di_copy, df_uniprot_3Di_merging_stats
    else:
        return df_uniprot_3Di_copy


def preprocess_3di_fasta(fasta_path: str, accessions_whitelist: list):

    fasta_data_3di = read_fasta(
        fasta_path
    )  # only contains four model organisms for now

    df_uniprot_3Di = pd.DataFrame.from_records(
        process_fasta(fasta_data_3di=fasta_data_3di),
        columns=["Uniprot", "fold_number", "sequence3Di"],
    ).sort_values(
        ["Uniprot", "fold_number"]
    )  # , index="Uniprot"

    # Only keep those that are in our sequence dataset!
    df_uniprot_3Di = df_uniprot_3Di[df_uniprot_3Di.Uniprot.isin(accessions_whitelist)]

    # also returns stats df with sequence identity between merged structures
    df_uniprot_3Di_merged, df_uniprot_3Di_merging_stats = merge_subsequences(
        df_uniprot_3Di=df_uniprot_3Di
    )

    df_uniprot_3Di_merged = df_uniprot_3Di_merged.assign(
        len_matches_aa_sequence=df.sequence.loc[df_uniprot_3Di_merged.index]
        .str.len()
        .values
        == df_uniprot_3Di_merged.sequence3Di.str.len().values
    )
    assert ~df_uniprot_3Di_merged.index.duplicated().any()
    return df_uniprot_3Di_merged, df_uniprot_3Di_merging_stats

In [16]:
df_uniprot_3Di, df_uniprot_3Di_merging_stats = preprocess_3di_fasta(
    RAW_DATA_FOLDER + "alphafolddb/3Di_sequences.fasta",
    accessions_whitelist=df.index.tolist(),
)
median_merging_overlap = df_uniprot_3Di_merging_stats.merging_overlaps_seq_ident.apply(
    np.median
).median()
mean_merging_overlap = df_uniprot_3Di_merging_stats.merging_overlaps_seq_ident.apply(
    np.mean
)
save_data(df_uniprot_3Di, "3Di_alphafold4", folder_path=DATASET_FOLDER)
df_uniprot_3Di.head()

,sequence3Di,len_matches_aa_sequence
Uniprot,,
A2VEC9,dddpvvvvpppppdapaakdkdkdkdkdwafpfdkdkdkdflvcvv...,True
A4UGR9,dddddddddddddddddddddddddddddddddddddddddddpvv...,True
O14686,dddadddddddddddddddddddddddddddddddddddddddddd...,True
O15018,dfpdlvclvlvlvllvvvlvvvvvvddpdplnvlsvllsvvsnvvs...,True
O15050,dddppppddqllvqlvvlqvqlvvcvvvvnlvsslvslvvscvspv...,True


## GOA

In [13]:
uniprot_accessions = set(df.index.unique())

df_go_ebi = pd.DataFrame()

print("Filtering GO annotations...", end=" ")
with pd.read_table(
    RAW_DATA_FOLDER+"goa/goa_uniprot_all_ebi_filtered.tsv.gz",
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
    dtype={
        "Uniprot": "string",
        "qualifier": "category",
        "go_id": "string",
        "evidence_code": "category",
        "aspect": "category",
        "date": "int",
    },
    chunksize=10**6,
    # parse_dates=["date"],
) as reader:
    for df_chunk in reader:
        df_chunk_filtered = (
            df_chunk[df_chunk.Uniprot.isin(uniprot_accessions)]
            .drop("date", axis=1)
            .drop_duplicates()
        )
        df_go_ebi = pd.concat([df_go_ebi, df_chunk_filtered], axis=0).reset_index(
            drop=True
        )
print("done")

Filtering GO annotations... done


In [14]:
df_go_ebi = df_go_ebi.drop_duplicates()

In [ ]:
# Using numbers starting with 0 as index halves the file size!
df_go_ebi = df_go_ebi.reset_index(drop=True)

save_data(df_go_ebi, "goa", DATASET_FOLDER)

## GO

In [ ]:
import obonet
graph_go = obonet.read_obo(RAW_DATA_FOLDER+"ontologies/go.obo")

save_data(graph_go, "go_obo", folder_path=DATASET_FOLDER)